In [ ]:
from queue import Queue

class Broadcaster:
    def __init__(self, destinations, queue) -> None:
        self.destinations = destinations
        self.name = "broadcaster"
        self.queue = queue

    def receivePulse(self, value, _):
        for dest in self.destinations:
            self.queue.put((value, dest, self.name))

class FlipFlop:
    def __init__(self, name, destinations, queue) -> None:
        self.name = name
        self.state = 0
        self.destinations = destinations
        self.queue = queue

    def receivePulse(self, value, source):
        if value != 1:
            self.state = 1-self.state
            for dest in self.destinations:
                self.queue.put((self.state, dest, self.name))

class Conjunction:
    def __init__(self, name, destinations, queue) -> None:
        self.name = name
        self.destinations = destinations
        self.queue = queue
        self.memory = {}

    def addInput(self, input):
        self.memory[input] = 0

    def receivePulse(self, value, source):
        self.memory[source] = value
        pulse = 0
        for p in self.memory.values():
            if p == 0:
                pulse = 1
                break
        for dest in self.destinations:
            self.queue.put((pulse, dest, self.name))


f = open("input.txt", "r")

modules = {}
conjunction_modules = {}
pulse_queue = Queue()

for line in f:
    [module, destinations] = line.replace('\n', '').replace(' ', '').split("->")
    destinations = [d for d in destinations.split(',') if d != ''] 
    if module.startswith('%'):
        name = module[1:]
        modules[name] = FlipFlop(name, destinations, pulse_queue)
    elif module.startswith('&'):
        name = module[1:]
        new_module = Conjunction(name, destinations, pulse_queue)
        modules[name] = new_module
        conjunction_modules[name] = new_module
    elif module == "broadcaster":
        modules["broadcaster"] = Broadcaster(destinations, pulse_queue)

for conjunction_module in conjunction_modules.values():
    for module in modules.values():
        if conjunction_module.name in module.destinations:
            conjunction_module.addInput(module.name)

def pressButton(nb_of_pulses):
    # Pulse = (value, destination, source)
    pulse_queue.put((0, "broadcaster", "button"))

    while not pulse_queue.empty():
        pulse = pulse_queue.get()
        value = pulse[0]
        destination = pulse[1]
        source = pulse[2]
        nb_of_pulses[value]+=1
        # print(f"Sending pulse {value} from {source} to {destination}")
        if destination == "output":
            continue
        elif destination in modules:
            modules[destination].receivePulse(value, source)

nb_of_pulses = [0, 0]

for _ in range(1000):
    pressButton(nb_of_pulses)

print(nb_of_pulses)
print(nb_of_pulses[0]*nb_of_pulses[1])



In [ ]:
from collections import defaultdict
import math
from queue import Queue

class Broadcaster:
    def __init__(self, destinations, queue) -> None:
        self.destinations = destinations
        self.name = "broadcaster"
        self.queue = queue

    def receivePulse(self, value, _):
        for dest in self.destinations:
            self.queue.put((value, dest, self.name))

class FlipFlop:
    def __init__(self, name, destinations, queue) -> None:
        self.name = name
        self.state = 0
        self.destinations = destinations
        self.queue = queue

    def receivePulse(self, value, source):
        if value != 1:
            self.state = 1-self.state
            for dest in self.destinations:
                self.queue.put((self.state, dest, self.name))

class Conjunction:
    def __init__(self, name, destinations, queue) -> None:
        self.name = name
        self.destinations = destinations
        self.queue = queue
        self.memory = {}

    def addInput(self, input):
        self.memory[input] = 0

    def receivePulse(self, value, source):
        self.memory[source] = value
        pulse = 0
        for p in self.memory.values():
            if p == 0:
                pulse = 1
                break
        for dest in self.destinations:
            self.queue.put((pulse, dest, self.name))


f = open("input.txt", "r")

modules = {}
conjunction_modules = {}
pulse_queue = Queue()

for line in f:
    [module, destinations] = line.replace('\n', '').replace(' ', '').split("->")
    destinations = [d for d in destinations.split(',') if d != ''] 
    if module.startswith('%'):
        name = module[1:]
        modules[name] = FlipFlop(name, destinations, pulse_queue)
    elif module.startswith('&'):
        name = module[1:]
        new_module = Conjunction(name, destinations, pulse_queue)
        modules[name] = new_module
        conjunction_modules[name] = new_module
    elif module == "broadcaster":
        modules["broadcaster"] = Broadcaster(destinations, pulse_queue)

for conjunction_module in conjunction_modules.values():
    for module in modules.values():
        if conjunction_module.name in module.destinations:
            conjunction_module.addInput(module.name)

# Keep track of the number of times the button has to be pressed for each module to send a high pulse to vf (which feeds rx)
button_presses_to_high = {'pm': defaultdict(int), 'mk': defaultdict(int), 'pk': defaultdict(int), 'hf': defaultdict(int)}

def pressButton(nb_of_pulses, nb_of_button_presses):
    pulse_count = 0
    # Pulse = (value, destination, source)
    pulse_queue.put((0, "broadcaster", "button"))

    while not pulse_queue.empty():
        pulse_count += 1
        pulse = pulse_queue.get()
        value = pulse[0]
        destination = pulse[1]
        source = pulse[2]
        nb_of_pulses[value]+=1
        if destination == "output":
            continue
        elif destination in modules:
            modules[destination].receivePulse(value, source)
        elif destination == "rx":
            vf_module = modules[source]
            for parent_m, parent_v in vf_module.memory.items():
                if parent_v == 1:
                    button_presses_to_high[parent_m][nb_of_button_presses] += 1

nb_of_pulses = [0, 0]
nb_of_button_presses = 0

for _ in range(40000):
    nb_of_button_presses += 1
    pressButton(nb_of_pulses, nb_of_button_presses)

# 40000 button presses are enough to show us that there is a cycle for each module
print(button_presses_to_high)

# Now we just need to find the lcm of the cycle lengths for these 4 modules
cycle_lengths = []
for module in button_presses_to_high:
    for cycle_length in button_presses_to_high[module].keys():
        cycle_lengths.append(cycle_length)
        break
print(cycle_lengths)
print(math.lcm(*cycle_lengths))

